In [1]:
print("OK")

OK


In [5]:
%pwd

'c:\\Users\\KIIT0001\\Documents\\Medical_chatbot\\research'

In [1]:
import os
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\KIIT0001\\Documents\\Medical_chatbot'

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents
    

In [7]:
extract_data=load_pdf_file(data='Data/')

In [9]:
def text_split(extract_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extract_data)
    return text_chunks


In [10]:
text_chunks=text_split(extract_data)
print("Length of the text chunks:-", len(text_chunks))

Length of the text chunks:- 7023


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings


In [12]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [13]:
embeddings=download_hugging_face_embeddings()
print(embeddings)

c:\Users\KIIT0001\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [17]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
#import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-qbv17we.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [18]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [19]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [21]:
docsearch

In [22]:
retriever = docsearch.as_retriever(search_type="similarity", search_kewargs={"k":2})

In [23]:
retrieved_docs=retriever.invoke("What is swelling?")
retrieved_docs

[Document(id='62cdb36d-3779-416e-a719-760516373d55', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 72.0, 'page_label': '73', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\medical_book.pdf', 'total_pages': 759.0}, page_content='swelling. This red, painful region grows bigger as the\ninfection and resulting tissue damage spread. An untreat-\ned infection may spread to the lymphatic system (acute\nlymphangitis ), the lymph nodes ( lymphadenitis ), the\nbloodstream (bacteremia), or into deeper tissues. Celluli-\ntis most often occurs on the face, neck, and legs.\nOrbital cellulitis\nA very serious infection, called orbital cellulitis,\noccurs when bacteria enter and infect the tissues sur-'),
 Document(id='e5046dc0-228f-4a00-b890-ff06f364272d', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 519.0, 'page_label': '520', 'producer'

In [24]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os

In [25]:
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [26]:
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0
)

In [27]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)




prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [28]:
response = rag_chain.invoke({"input": "what is Cirrhosis?"})
print(response["answer"])

Cirrhosis is a liver disease characterized by scarring of the liver, which impairs its ability to remove toxins and perform other essential functions. It can be caused by various factors, including poor nutrition, alcoholism, and certain genetic disorders.
